# parsing+chunking

Парсинг страниц веб

In [6]:
import requests
from bs4 import BeautifulSoup
from langchain.schema import Document

# URL сайта
url = "https://www.eurochem.ru/"

try:
    # Загрузка HTML-страницы
    response = requests.get(url)
    response.raise_for_status()  # Проверка на ошибки

    # Парсинг HTML
    soup = BeautifulSoup(response.text, "html.parser")

    # Извлечение текста (удаляем теги)
    text = soup.get_text(separator="\n", strip=True)

    # Создание объекта Document для LangChain (опционально)
    doc = Document(page_content=text, metadata={"source": url})

    # Вывод текста
    print(doc.page_content)

except requests.exceptions.RequestException as e:
    print(f"Ошибка при загрузке страницы: {e}")

АО «Минерально-химическая компания ЕвроХим»
Истории
Другие сайты
Продажи удобрений
Свяжитесь с нами
Поиск
Поиск
Искать
О Компании
О Компании
Наши активы
Наши проекты
Корпоративное управление
Комплаенс
ПроТех Лаб
Продукция
Продукция
Удобрения и кормовые продукты
Промышленная продукция
Устойчивое развитие
Устойчивое развитие
ESG
Промышленная безопасность, охрана труда и экология
Сопровождение продукции
Карьера
Карьера
Жизнь в ЕвроХим
Специальные карьерные программы
Наши вакансии
Контакты HR
Пресс-центр
Пресс-центр
Все новости
Наш бренд
Мы в социальных сетях
Инвесторам
Инвесторам
Долговые инвесторы
Поставщикам
Поиск
Искать
­
Другие сайты
Корпоративные
EuroChem Group AG
Продажи
Минеральные удобрения
Промышленная и кормовая продукция
Раскрытие информации
АО «МХК «ЕвроХим»
Минерально-химическая компания ЕвроХим
Один из ведущих российских производителей минеральных удобрений. Мы предлагаем полный спектр решений для удовлетворения растущего спроса на продукты питания.
Купить удобрения
8 800 20

In [8]:
# Разделяем текст на строки
lines = text.split("\n")

# Фильтруем строки, оставляя только те, где больше 2 слов
filtered_text = [line.strip() for line in lines if len(line.split()) > 2]

# Выводим результат
for line in filtered_text:
    print(line)

АО «Минерально-химическая компания ЕвроХим»
Свяжитесь с нами
Удобрения и кормовые продукты
Промышленная безопасность, охрана труда и экология
Жизнь в ЕвроХим
Специальные карьерные программы
Мы в социальных сетях
EuroChem Group AG
Промышленная и кормовая продукция
АО «МХК «ЕвроХим»
Минерально-химическая компания ЕвроХим
Один из ведущих российских производителей минеральных удобрений. Мы предлагаем полный спектр решений для удовлетворения растущего спроса на продукты питания.
8 800 201-01-01
Единый контактный номер
Фокус на клиенте
Мы стремимся помочь фермерам повысить объём и качество урожая, чтобы удовлетворить потребности растущего населения
Широкий продуктовый портфель
Мы предлагаем полный спектр удобрений, от стандартных продуктов до премиальных решений с улучшенными характеристиками и повышенной эффективностью
Мы стремимся быть социально-ответственным бизнесом во всех аспектах: как работодатель, сосед, клиент или поставщик
Защита персональных данных
Политика в отношении файлов cook

In [9]:
# Находим индекс строки "Политика в отношении файлов cookie"
cookie_policy_index = next((i for i, line in enumerate(filtered_text) if "Политика в отношении файлов cookie" in line), None)

# Если найдено, отсекаем всё после этой строки
if cookie_policy_index is not None:
    filtered_text = filtered_text[:cookie_policy_index]

In [14]:
# Выводим результат
for line in filtered_text:
    print(line)
    print('')

АО «Минерально-химическая компания ЕвроХим»

Свяжитесь с нами

Удобрения и кормовые продукты

Промышленная безопасность, охрана труда и экология

Жизнь в ЕвроХим

Специальные карьерные программы

Мы в социальных сетях

EuroChem Group AG

Промышленная и кормовая продукция

АО «МХК «ЕвроХим»

Минерально-химическая компания ЕвроХим

Один из ведущих российских производителей минеральных удобрений. Мы предлагаем полный спектр решений для удовлетворения растущего спроса на продукты питания.

8 800 201-01-01

Единый контактный номер

Фокус на клиенте

Мы стремимся помочь фермерам повысить объём и качество урожая, чтобы удовлетворить потребности растущего населения

Широкий продуктовый портфель

Мы предлагаем полный спектр удобрений, от стандартных продуктов до премиальных решений с улучшенными характеристиками и повышенной эффективностью

Мы стремимся быть социально-ответственным бизнесом во всех аспектах: как работодатель, сосед, клиент или поставщик

Защита персональных данных



Итоговая функция:

In [7]:
import requests
from bs4 import BeautifulSoup

def parse_url(url):
    try:
        # Загрузка HTML-страницы
        response = requests.get(url)
        response.raise_for_status()  # Проверка на ошибки

        # Парсинг HTML
        soup = BeautifulSoup(response.text, "html.parser")

        # Извлечение текста (удаляем теги)
        text = soup.get_text(separator="\n", strip=True)

        # Разделяем текст на строки
        lines = text.split("\n")

        # Фильтруем строки, оставляя только те, где больше 2 слов
        filtered_text = [line.strip() for line in lines if len(line.split()) > 2]

        # Находим индекс строки "Политика в отношении файлов cookie"
        cookie_policy_index_1 = next((i for i, line in enumerate(filtered_text) if "Политика в отношении файлов cookie" in line), None)

        # Находим индекс строки "Мы используем cookie"
        cookie_policy_index_2 = next((i for i, line in enumerate(filtered_text) if "Мы используем cookie" in line), None)

        # Находим минимальное значение из индексов
        if cookie_policy_index_1 is not None or cookie_policy_index_2 is not None:
            # Выбираем минимальный индекс
            cutoff_index = min(filter(lambda x: x is not None, [cookie_policy_index_1, cookie_policy_index_2]))

            # Отсекаем всё после найденного индекса
            filtered_text = filtered_text[:cutoff_index]

        result_string = "\n".join(filtered_text)

        return result_string
        
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при загрузке страницы: {e}")
        return 'error'

In [9]:
res = parse_url("https://digtp.com/contacts")

print(res)

115054, г. Москва, ул. Дубининская, д. 57, стр. 1а
Маршрут на Яндекс картах
Директор по персоналу
Взаимодействие со СМИ
Руководитель по корпоративным коммуникациям
Начальник управления потребностью ИТ
для промышленности и прочих сфер.
Политики и документы
ООО «Цифровые технологии и платформы»
является аккредитованной ИТ-компанией,
свидетельство № АО-20211230-3269820362-3 от 10.01.2022
